<a href="https://colab.research.google.com/github/francji1/01RAD/blob/main/python/01RAD_Ex04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01RAD Exercise 05

Today's exercise
 * Numerical Linear Algebra view-
 * Residuals


In [ ]:
!pip install rpy2
%load_ext rpy2.ipython

In [ ]:
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr

# Activate the automatic conversion between R and pandas DataFrame
pandas2ri.activate()


In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
import matplotlib.pyplot as plt

from statsmodels.datasets import get_rdataset
from scipy.stats import t,norm


%matplotlib inline

In [ ]:
# Use the %%R cell magic to run R code and retrieve the Gasoline dataset
#%%R -o df
#install.packages("RSADBE")
#library(RSADBE)
#data(gasoline)
#df <- gasoline

In [ ]:
# Reading the data from the URL
data_url = "https://dasl.datadescription.com/download/data/3096"
df = pd.read_csv(data_url, delimiter="\t")
df

In [ ]:
def get_regression(X, Y):
    """
    Calculate linear regression coefficients, standard errors,
     t-values, p-values, and 95% confidence intervals.

    Parameters:
    - X: DataFrame of independent variables.
    - Y: DataFrame of dependent variable.

    Returns:
    - DataFrame with coefficients, standard errors, t-values, p-values,
      and 95% confidence intervals.
    """

    # Copy data to avoid modifying original data
    X = pd.DataFrame(X.copy())
    Y = pd.DataFrame(Y.copy())

    # Add constant for intercept
    X['const'] = 1
    X = X[['const'] + [col for col in X if col != 'const']]

    # QR Decomposition
    # Q, R = np.linalg.qr(X.values)

    # Calculate regression coefficients using the formula
    beta_hat = np.linalg.inv(X.values.T @ X.values) @ X.values.T @ Y.values

    # Calculate regression coefficients using corrected QR decomposition formula
    #beta_hat = np.linalg.inv(R) @ (Q.T @ Y.values)[:R.shape[0]]

    # Predicted values and residuals
    Y_pred = X.values @ beta_hat
    residuals = Y_pred - Y.values
    #Residual Sum of Squares (RSS)
    RSS = residuals.T @ residuals
    print(RSS)
    MSE = RSS/Y.shape[0]
    print(MSE)

    # Adjusted degrees of freedom
    df = Y.shape[0] - X.shape[1] - 1
    # Estimate of error variance (RSS divided by the degrees of freedom of the residuals)
    sigma2_hat = (RSS[0][0] / df)
    print(sigma2_hat)
    # Standard errors of coefficients
    se_beta_hat = np.sqrt(sigma2_hat * np.diag(np.linalg.inv(X.values.T @ X.values)))

    # t-values and p-values
    t_values = beta_hat.reshape(-1) / se_beta_hat

    p_values = 2 * (1 - t.cdf(np.abs(t_values), df))

    # Critical t-value for 95% CI
    alpha = 0.05
    t_critical = t.ppf(1 - alpha/2, df)

    # 95% Confidence Intervals
    ci_lower = beta_hat.reshape(-1) - t_critical * se_beta_hat
    ci_upper = beta_hat.reshape(-1) + t_critical * se_beta_hat

    # Return results as a DataFrame
    return pd.DataFrame({
        'coef': beta_hat.reshape(-1),
        'std err': se_beta_hat,
        't': t_values,
        'P > |t|': p_values,
        '95% CI Lower': ci_lower,
        '95% CI Upper': ci_upper
    }, index=X.columns)


In [ ]:
# Selecting MPG as Y and HP as X
Y = df[['MPG']]
X = df[['Horsepower']]

# Using the get_regression function to get results
results = get_regression(X, Y)
results


In [ ]:
# Using statsmodels
model_sm = sm.OLS(Y, sm.add_constant(X))
results_sm = model_sm.fit()
print(results_sm.summary())

In [ ]:
# Compute residuals
residuals = results_sm.resid

# Compute and print statistics
print("Mean of residuals:", residuals.mean())
print("Standard deviation of residuals:", residuals.std())
print("Variance of residuals:", residuals.var())
print("Scaled deviance of residuals:", (residuals**2).sum() / (len(df) - 2))


In [ ]:
# Plotting results
fig, ax = plt.subplots(figsize=(12, 8))
fig = sm.graphics.plot_fit(results_sm, 1, ax=ax)
plt.show()

In [ ]:
# Plot residuals as a histogram
plt.hist(residuals, bins=20, edgecolor='k', alpha=0.65)
plt.title('Histogram of Residuals')
plt.xlabel('Residual')
plt.ylabel('Frequency')
plt.show()

# Q-Q plot of residuals
stats.probplot(residuals, plot=plt)
plt.title('Q-Q Plot of Residuals')
plt.show()

In [ ]:
results_sm

In [ ]:
    X['const'] = 1
    X

In [ ]:
X = X[['const'] + [col for col in X if col != 'const']]

In [ ]:
X

In [ ]:
# Convert X to a numpy array for matrix operations
X_np = X.values

# Compute X^T X
XtX = X_np.T @ X_np

# Calculate the hat matrix H
H = X_np @ np.linalg.inv(XtX) @ X_np.T

print("Dimensions of H:", H.shape)
print("Dimensions of X:", X_np.shape)

In [ ]:
H

In [ ]:
# Eigenvalues of H
eigenvalues = np.linalg.eigvals(H)
print("Eigenvalues of H:", np.round(eigenvalues, 10))

# Check if H is idempotent
idempotent_diff = np.sum(np.round(H @ H - H, 5))
print("Difference between H^2 and H:", idempotent_diff)

# Check if H is symmetric
symmetry_diff = np.round(H.T - H, 10)
print("Difference between H^T and H:", symmetry_diff)

# Dimensions
print("Dimensions of H:", H.shape)
print("Dimensions of X:", X.shape)

# Predicted values
hat_Y = H @ Y


**Task:** In the simple linear regression model, construct a Wald test for $H_0 : \beta_1 = 17 \beta_0$ versus $H_1 : \beta_1 \neq 17 \beta_0$.

**Solution**.  Let $\delta = \beta_1 - 17 \beta_0$.  The MLE is $\hat{\delta} = \hat{\beta}_1 - 17 \hat{\beta}_0$, with estimated standard error $\hat{\text{se}}(\hat{\delta})$, where

$$\hat{\text{se}}(\hat{\delta})^2 = \hat{\text{se}}(\hat{\beta}_1 - 17 \hat{\beta}_0)^2 = \hat{\text{se}}(\hat{\beta}_1)^2 + 17^2 \hat{\text{se}}(\hat{\beta}_0)^2 $$

and the estimates for the parameter standard deviations are


$$
\hat{\text{se}}(\hat{\beta}_0) = \frac{\hat{\sigma}}{s_X \sqrt{n}} \sqrt{\frac{\sum_{i=1}^n X_i^2}{n}}
\quad \text{and} \quad
\hat{\text{se}}(\hat{\beta}_1) = \frac{\hat{\sigma}}{s_X \sqrt{n}}
$$

The Wald test then checks if $|W| < z_{\alpha / 2}$, where

$$W = \frac{\hat{\delta} - 0}{\hat{\text{se}}(\hat{\delta})}
= \frac{\hat{\beta}_1 - 17 \hat{\beta}_0}{\sqrt{\hat{\text{se}}(\hat{\beta}_1)^2 + 17^2 \hat{\text{se}}(\hat{\beta}_0)^2}}$$

Exercise: Test if regression coefficient for Intercept = -200 time regression coefficient for Horsepower




In [ ]:
results_sm.params

In [ ]:
results_sm.bse

In [ ]:
(results_sm.params[0]/results_sm.params[1])**2

In [ ]:
W = (results_sm.params[0]+170*results_sm.params[1])/np.sqrt(results_sm.bse[0]**2 + (170**2)*results_sm.bse[1]**2)
W

In [ ]:
p_value = 2*(1-norm.cdf(abs(W)))
p_value

**Task**
* compute fitted values manually by the help of original data set and sm model coeff
* compare results with output from the fitted values from sm object
* compare with result obtained by H matrix


In [ ]:
(X @ results_sm.params).mean()

In [ ]:
results_sm.fittedvalues.mean()

In [ ]:
(H @ Y).mean()

In [ ]:
M = np.identity(H.shape[0]) - H
e = (M @ Y)
e

In [ ]:
results_sm.resid

**Task**
* Check if the eigenvalues of H  consist of r ones and n-r zeros.
* Check if the eigenvalues of M consist  of n-r ones and r  zeros. (M = I -H)
* Check if H and M are idempotent

In [ ]:
eigenvalues

In [ ]:
np.linalg.eigvals(M)

## Residual  Analysis (In R need to add %%R)

In [ ]:
# Set random seed for reproducibility
np.random.seed(21)

# Generate data
n = 60
X0 = np.ones(n)
X1 = np.random.uniform(10, 40, n)
X = np.column_stack((X0, X1))
e = np.random.normal(0, 2, n)
beta = np.array([4, 2])
Y = X @ beta + e

df_m1 = pd.DataFrame({
    'Y': Y,
    'X': X1
})

print(df_m1.head())

In [ ]:
%%R
# Summary and visualisation of our dataset
# Let's generate some data
set.seed(21)
n    <- 50
X0   <- rep(1,n)
X1   <- runif(n,10,40)
X    <- cbind(X0,X1)
e    <- rnorm(n,0,2)
beta <- c(4,2)
Y    <- X%*%matrix(beta) + e
df_m1 <- data.frame(Y = Y, X = X1)

summary(df_m1)


In [ ]:
%%R
# OLS estimation of regression coefficients
m1 <- lm(Y ~ X, df_m1)
summary(m1)


In [ ]:
print(results_sm.summary())

In [ ]:
%%R

# Fast post-hoc analysis of residuals
opar <- par(mfrow=c(2,2))
plot(m1)
par(opar)



In [ ]:
# Plotting results
fig, ax = plt.subplots(figsize=(12, 8))
fig = sm.graphics.plot_fit(results_sm, 1, ax=ax)
plt.show()

In [ ]:
# Create a 2x2 grid of subplots
fig, axes = plt.subplots(2, 2, figsize=(10, 8))

# Residuals vs. Fitted Values
axes[0, 0].scatter(results_sm.fittedvalues, results_sm.resid)
axes[0, 0].set_xlabel("Fitted Values")
axes[0, 0].set_ylabel("Residuals")
axes[0, 0].set_title("Residuals vs. Fitted Values")

# Quantile-Quantile Plot of Residuals
sm.qqplot(results_sm.resid, line='s', ax=axes[0, 1])
axes[0, 1].set_title("Q-Q Plot of Residuals")

# Standardized Residuals vs. Fitted Values
axes[1, 0].scatter(results_sm.fittedvalues, results_sm.get_influence().resid_studentized_internal)
axes[1, 0].set_xlabel("Fitted Values")
axes[1, 0].set_ylabel("Standardized Residuals")
axes[1, 0].set_title("Standardized Residuals vs. Fitted Values")

# Cook's Distance Plot
sm.graphics.plot_leverage_resid2(results_sm, ax=axes[1, 1])
axes[1, 1].set_title("Cook's Distance Plot")

# Adjust layout
plt.tight_layout()

# Show the plots
plt.show()

In [ ]:
%%R

hist(resid(m1))
fitted(m1)


In [ ]:
%%R

#residuals vs. fitted
plot(residuals(m1) ~ fitted(m1))



In [ ]:

#########################################
# How R computes different residuals?
# R provides built-in functions rstandard() and rstudent() as as part of influence.measures
# They re-normalize the residuals to have unit variance,
# using an overall and leave-one-out measure of the error variance respectively.


In [ ]:
%%R

# manual computation of standartized residuals : help(rstandard)
# R standartized residual is ordinary residual divided by an estimate of its standard deviation:
r_standard = residuals(m1)/(sigma(m1)*sqrt(1-hatvalues(m1)))
# comparison with built-in function
r_standard - rstandard(m1)



In [ ]:
%%R
#Studentized residuals: "externally studentized residuals" or "deleted residuals,"
#Studentized residuals for any given data point are computed from a model fit
# to every other data point except the one in question.

# compare with slide 1, lecture 5
anova(m1)

A = sqrt(1 - (1/n + (X1-mean(X1))^2/sum((X1-mean(X1))^2)))
r_stud <- residuals(m1) / (sigma(m1)*A)

round(rstudent(m1) - r_stud,4)
round(rstandard(m1) - r_stud,4)

rstandard(m1, type = "predictive")
# we will come back to standardized and studentized residuals later
# in outlier diagnostic session




In [ ]:
%%R
install.packages("tidyverse")
install.packages("ggpubr")
library(tidyverse)
library(ggpubr)

# Plot with ggplot, add residuals to dataset

df_m1 <- df_m1  %>%
  mutate(predicted = predict(m1),
         residuals = residuals(m1),
         rstandard = rstandard(m1))

p1<-ggplot(data=df_m1,mapping=aes(x=predicted,y=residuals)) +
  geom_point() +
  geom_hline(yintercept=0,linetype="dashed")

p2<-ggplot(data=df_m1,mapping=aes(x=Y,y=residuals)) +
  geom_point() +
  geom_hline(yintercept=0,linetype="dashed")

p3<-ggplot(data=df_m1,mapping=aes(x=predicted,y=rstandard)) +
  geom_point() +
  geom_hline(yintercept=0,linetype="dashed")

p4<-ggplot(data=df_m1,mapping=aes(x=Y,y=rstandard)) +
  geom_point() +
  geom_hline(yintercept=0,linetype="dashed")

ggarrange(p1, p2, p3, p4, labels = c("A", "B","C","D"), common.legend = TRUE, legend = "bottom")



In [ ]:
%%R

# Visualization of residuals (orthogonal distance to be minimized by OLS method)

ggplot(df_m1, aes(x = X, y = Y)) +
  geom_smooth(method = "lm", se = FALSE, color = "lightgrey") +
  geom_segment(aes(xend = X, yend = predicted), alpha = .2) +
  geom_point(aes(color = residuals)) +
  scale_color_gradient2(low = "blue", mid = "white", high = "red") +  # Colors to use here
  # geom_point() +
  # guides(color = FALSE) +
  geom_point(aes(y = predicted), shape = 1) +
  theme_bw()



Lets play with regression model and obtain different shapes of residuals

In [ ]:
%%R

X1   <- runif(n,0,1)
X    <-cbind(X0,sin(2*pi*X1))
Y    <- X%*%matrix(c(1,1)) + rnorm(n,0,(0.1)^2)
df_m1 <- data.frame(Y = Y, X = X1)
summary(df_m1)
ggpairs(df_m1)
m1 <- lm(Y ~ X, df_m1)
opar <- par(mfrow=c(2,2))
plot(m1)
par(opar)



In [ ]:
%%R

X1   <- runif(n,0,10)
X    <- cbind(X0,log(0.4 + X1))
Y    <- X%*%matrix(c(1,1)) + rnorm(n,0,0.1)
df_m1<- data.frame(Y = Y, X = X1)
ggpairs(df_m1)
m1 <- lm(Y ~ X, df_m1)
opar <- par(mfrow=c(2,2))
plot(m1)
par(opar)


In [ ]:
%%R

X0   <- rep(1,n)
X1   <- runif(n,10,40)
X    <- cbind(X0,X1)
Y   <- X%*%matrix(beta) + (rnorm(n,0,(X1/10)^2)+X1/10)
df_m1 <- data.frame(Y = Y, X = X1)
ggpairs(df_m1)
summary(df_m1)
m1 <- lm(Y ~ X, df_m1)
opar <- par(mfrow=c(2,2))
plot(m1)
par(opar)
# repeat plots ;)